In [65]:
import json
import pandas as pd
# lista di eventi di gioco fornita in formato json
players = json.load(open('../players.json'))
events = json.load(open('../worldCup-final.json'))

In [2]:
df_players = pd.DataFrame(players)

In [3]:
df_players = df_players.set_index('playerId')

In [4]:
df = pd.DataFrame(events)

def transform_pos(x):
    if len(x) < 2:
        x.append(x[0])
    return x

df['positions'] = df['positions'].apply(transform_pos)

df['x_start'] = [p[0]['x'] for p in df['positions']]
df['y_start'] = [p[0]['y'] for p in df['positions']]
df['x_end'] = [p[1]['x'] for p in df['positions']]
df['y_end'] = [p[1]['y'] for p in df['positions']]

df = df[df['playerId'] != 0] # rimuovere il signor arbitro (playerId = 0)

df.head()

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,positions,subEventId,subEventName,tags,teamId,x_start,y_start,x_end,y_end
0,8,Pass,1.892339,263883958,2058017,1H,14943,"[{'y': 51, 'x': 50}, {'y': 51, 'x': 50}]",85,Simple pass,[],Croatia,50,51,50,51
1,8,Pass,3.889375,263883959,2058017,1H,69968,"[{'y': 52, 'x': 39}, {'y': 74, 'x': 34}]",85,Simple pass,[{'id': 1801}],Croatia,39,52,34,74
2,8,Pass,6.140946,263883960,2058017,1H,8287,"[{'y': 74, 'x': 34}, {'y': 93, 'x': 30}]",85,Simple pass,[{'id': 1801}],Croatia,34,74,30,93
3,8,Pass,9.226570,263883963,2058017,1H,69409,"[{'y': 93, 'x': 30}, {'y': 65, 'x': 11}]",85,Simple pass,[{'id': 1801}],Croatia,30,93,11,65
4,8,Pass,12.658969,263883964,2058017,1H,135747,"[{'y': 65, 'x': 11}, {'y': 51, 'x': 64}]",84,Launch,[{'id': 1801}],Croatia,11,65,64,51


In [5]:
df_teams = df[['playerId','teamId']]

In [6]:
df_teams = df_teams.drop_duplicates()

In [7]:
df_teams = df_teams.set_index('playerId')

In [8]:
df_teams

,teamId
playerId,
14943,Croatia
69968,Croatia
8287,Croatia
69409,Croatia
135747,Croatia
3309,France
14812,Croatia
3476,Croatia
105361,Croatia


In [16]:
df_player_period = list(df.groupby(['playerId', 'matchPeriod']))

In [17]:
avgOverPeriod = []
totTimeDf = []
for pl in df_player_period:
    actions = pl[1]['eventSec'].sort_values(ascending=True)
    avgOverPeriod.append({'identificativo_calciatore':pl[0][0],'matchPeriod':pl[0][1],
                          'tempo_medio_tra_eventi':actions.diff().mean()
                        })
    totTimeDf.append({'identificativo_calciatore':pl[0][0],'matchPeriod':pl[0][1],
                              'totTime':(actions.iloc[-1] - actions.iloc[0])
                        })

In [18]:
df_avg_between_events = pd.DataFrame(avgOverPeriod).groupby('identificativo_calciatore').mean()
df_totTime = pd.DataFrame(totTimeDf).groupby('identificativo_calciatore').sum()

In [20]:
df_totTime

,totTime
identificativo_calciatore,
3309,5853.180274
3476,5790.695974
3682,5582.411102
7936,5619.238812
8200,2303.157857
8287,5673.205288
14812,5681.801226
14943,5624.998725
25381,5670.846313


In [21]:
mask = df_totTime['totTime'] <= 2700
nonConsiderati = list(df_totTime[mask].index)

In [22]:
#df_tempi = df_tempi[~mask]

In [23]:
#df_tempi.shape

In [24]:
nonConsiderati

[8200, 28115, 69411, 135810, 209091]

In [18]:
#df = df[~df['playerId'].isin(nonConsiderati)]

In [25]:
df_pos_media = pd.DataFrame(df.playerId)
df_pos_media = df_pos_media[~df_pos_media['playerId'].isin(nonConsiderati)]

In [26]:
df_pos_media['posizione_media_x'] = (df['x_start'] + df['x_end']) / 2
df_pos_media['posizione_media_y'] = (df['y_start'] + df['y_end']) / 2
df_pos_media.head()

data_grouped = df_pos_media.groupby('playerId')
posizione_media = data_grouped.mean()


In [27]:
df_posizione_media = posizione_media[['posizione_media_x','posizione_media_y']]
df_posizione_media

,posizione_media_x,posizione_media_y
playerId,,
3309,30.865854,63.000000
3476,58.303738,32.742991
3682,57.969231,41.592308
7936,44.371212,61.818182
8287,58.431373,70.259804
14812,67.436620,45.429577
14943,66.658730,58.888889
25381,27.773810,40.440476
25393,43.987952,65.433735


In [28]:
df_posizione_media.shape

(22, 2)

In [29]:
positions = ['x_start', 'x_end', 'y_start', 'y_end']
variances = df.groupby('playerId')[positions].var(ddof=0).sum(axis=1)
df_quad = pd.DataFrame(variances, columns=['distanza_quadratica_media'])
df_quad

,distanza_quadratica_media
playerId,
3309,1737.162403
3476,2209.508953
3682,2738.565680
7936,2507.257346
8200,2017.880000
8287,1611.544694
14812,3455.948423
14943,2251.700680
25381,1754.343537


In [30]:
df_quad.shape

(27, 1)

In [38]:
df_players.shape, df_teams.shape,df_posizione_media.shape, df_quad.shape, df_avg_between_events.shape

((25, 1), (27, 1), (22, 2), (27, 1), (27, 1))

In [83]:
#df_output = df_teams.join(df_posizione_media,how='outer').join(df_quad,how='outer').join(df_tempi,how='outer')

In [56]:
df_output = pd.concat([df_players,df_teams,df_posizione_media,df_quad,df_avg_between_events],axis=1)

In [57]:
#df_output.dropna(axis=0,inplace=True)

In [59]:
df_output.reset_index(level=0,inplace=True)
df_output

,index,name,teamId,posizione_media_x,posizione_media_y,distanza_quadratica_media,tempo_medio_tra_eventi
0,3309,R. Varane,France,30.865854,63.000000,1737.162403,152.832913
1,3476,I. Rakitić,Croatia,58.303738,32.742991,2209.508953,56.404163
2,3682,A. Griezmann,France,57.969231,41.592308,2738.565680,125.968276
3,7936,P. Pogba,France,44.371212,61.818182,2507.257346,100.902427
4,8200,S. N'Zonzi,France,NaN,NaN,2017.880000,121.218835
5,8287,L. Modrić,Croatia,58.431373,70.259804,1611.544694,56.760866
6,14812,I. Perišić,Croatia,67.436620,45.429577,3455.948423,83.731080
7,14943,M. Mandžukić,Croatia,66.658730,58.888889,2251.700680,92.220442
8,25381,H. Lloris,France,27.773810,40.440476,1754.343537,146.460852
9,25393,D. Lovren,Croatia,43.987952,65.433735,1903.351720,69.103012


In [62]:
df_output.rename(index=str, columns={"index":"identificativo_calciatore","name": "nome_calciatore", "teamId": "squadra_calciatore"},inplace=True)

In [63]:
df_output.to_csv('problema_1.csv',index=False)

In [64]:
df_output

,identificativo_calciatore,nome_calciatore,squadra_calciatore,posizione_media_x,posizione_media_y,distanza_quadratica_media,tempo_medio_tra_eventi
0,3309,R. Varane,France,30.865854,63.000000,1737.162403,152.832913
1,3476,I. Rakitić,Croatia,58.303738,32.742991,2209.508953,56.404163
2,3682,A. Griezmann,France,57.969231,41.592308,2738.565680,125.968276
3,7936,P. Pogba,France,44.371212,61.818182,2507.257346,100.902427
4,8200,S. N'Zonzi,France,NaN,NaN,2017.880000,121.218835
5,8287,L. Modrić,Croatia,58.431373,70.259804,1611.544694,56.760866
6,14812,I. Perišić,Croatia,67.436620,45.429577,3455.948423,83.731080
7,14943,M. Mandžukić,Croatia,66.658730,58.888889,2251.700680,92.220442
8,25381,H. Lloris,France,27.773810,40.440476,1754.343537,146.460852
9,25393,D. Lovren,Croatia,43.987952,65.433735,1903.351720,69.103012
